# Functions

OpenAI's ChatCompletion API enables you to pass a list of `functions` for it to optionally call in service of a query. If it chooses to execute a function, either by choice or instruction, it will the function's name along with its formatted parameters for *you* to evaluate. The OpenAI schema accepts a JSON Schema representation of your functions.

Marvin includes first-class utilities for working with OpenAI's function API. These utilities are completely standalone: you're free to use them with or without Marvin's other components or framework. 

In [ ]:
from marvin.openai import openai_fn

@openai_fn
def add(x: int, y:int) -> str:
    return x + y

### Serialization

*Marvin* allows auto creation of JSON Schemas from functions:

In [ ]:
from marvin.openai import openai_fn

@openai_fn
def add(x: int, y:int) -> str:
    '''Adds two numbers together'''
    return x + y

add.schema

Returns its JSON Schema to use with OpenAI's function API.

In [ ]:
{
    "name": "add",
    "description": "Adds two numbers together",
    "parameters": {
        "x": {
            "type": "int",
            "description": null
        },
        "y": {
            "type": "int",
            "description": null
        },
        "required": ["x", "y"],
    }
}

### Querying

*Marvin* allows you to prompt functions in natural language. 

In [ ]:
from marvin.openai import openai_fn

@openai_fn
def add(x: int, y:int) -> str:
    '''Adds two numbers together'''
    return x + y

add.prompt([
{
    'role': 'system',
    'content': 'You are a helpful calculator.',
},
{
    'role': 'user',
    'content': 'What is 2 + 2?',
}])

Returns the keyword arguments needed to evaluate with OpenAI's SDK.

In [ ]:
{
    'messages': [
        {'role': 'system', 'content': 'You are a helpful calculator.'},
        {'role': 'user', 'content': 'What is 2 + 2?'}
    ],
    'functions': [
        {
            'name': 'add',
            'description': 'Adds two numbers together',
            'parameters': {
                'type': 'object',
                'properties': {
                    'x': {'title': 'X', 'type': 'integer'},
                    'y': {'title': 'Y', 'type': 'integer'}
                },
                'required': ['x', 'y']
            }
        }
    ],
    'function_call': {'name': 'add'}
}

By default, if you prompt a function directly as above, we set the function call equal to the name of function provided. This, in effect, forces OpenAI to use this function in service of your prompt. You can pass function_call = 'auto' to the prompt method to disable this default behavior.

### Function Registry

OpenAI's ChatCompletion API enables you to pass a list of `functions` for it to optionally call in service of a query. If it chooses to execute a function, either by choice or instruction, it will return a choice of function along with its formatted parameters for *you* to evaluate. The OpenAI schema accepts a JSON Schema representation of your functions.

Marvin supplies an inuitive API for serializing your functions for use with OpenAI's Function API. 

In [ ]:
from marvin.openai import OpenAIFunctionRegistry

registry = OpenAIFunctionRegistry()

@registry.register
def add(x: int, y: int) -> int:
    '''Adds two numbers together'''
    return x + y

@registry.register
def subtract(x: int, y: int) -> int:
    '''Substracts `y` from `x`'''
    return x - y

registry.schema

Which returns the necessary keyword arguments to pass to OpenAI's SDK. Note the default behavior has changed: function_call is correctly set to 'auto' to indicate to OpenAI that it's free to choose between your passed functions (or invoking no function at all). 

In [ ]:
{
  "functions": [
    {
      "name": "add",
      "description": 'Adds two numbers together',
      "parameters": {
        "type": "object",
        "properties": {
          "x": {
            "title": "X",
            "type": "integer"
          },
          "y": {
            "title": "Y",
            "type": "integer"
          }
        },
        "required": [
          "x",
          "y"
        ]
      }
    },
    {
      "name": "subtract",
      "description": 'Substracts `y` from `x`',
      "parameters": {
        "type": "object",
        "properties": {
          "x": {
            "title": "X",
            "type": "integer"
          },
          "y": {
            "title": "Y",
            "type": "integer"
          }
        },
        "required": [
          "x",
          "y"
        ]
      }
    }
  ],
  "function_call": "auto"
}

As with a single function, you can prompt an entire function registry. This is effectively telling OpenAI that you'd like an answer and that it's allowed to call any one of your functions in service of a response

In [ ]:
registry.prompt([
    {
        'role': 'system',
        'content': 'You are a helpful calculator.',
    },
    {
        'role': 'user',
        'content': '(10 + 123) - 123?',
    }
])

In [ ]:
{
  "messages": [
    {
      "role": "user",
      "content": "What is (10 + 123) - 123?"
    }
  ],
  "functions": [
    {
      "name": "add",
      "description": "Adds two numbers together",
      "parameters": {
        "type": "object",
        "properties": {
          "x": {
            "title": "X",
            "type": "integer"
          },
          "y": {
            "title": "Y",
            "type": "integer"
          }
        },
        "required": [
          "x",
          "y"
        ]
      }
    },
    {
      "name": "subtract",
      "description": "Substracts `y` from `x`",
      "parameters": {
        "type": "object",
        "properties": {
          "x": {
            "title": "X",
            "type": "integer"
          },
          "y": {
            "title": "Y",
            "type": "integer"
          }
        },
        "required": [
          "x",
          "y"
        ]
      }
    }
  ],
  "function_call": "auto"
}

#### Composability

Given two function routers, you can easily compose them. This let's you separately define, say, one registry devoted to accessing and processing one data source, and another devoted to accessing and processing another (with stricter permissions, perhaps). Including them is as straightfoward as calling `include` (which is, of course, idempotent).

In [ ]:
math = OpenAIFunctionRegistry()

arithmetic = OpenAIFunctionRegistry()

trigonometry = OpenAIFunctionRegistry()

@arithmetic.register
def add(x: int, y: int) -> int:
    '''Adds two numbers together'''

@trigonometry.register
def tan(theta: float) -> float:
    '''Calculates the tangent of `theta`.'''
    return x - y

math.include(arithmetic)
math.include(trigonometry)

math.schema

In [ ]:
{
  "functions": [
    {
      "name": "add",
      "description": "Adds two numbers together",
      "parameters": {
        "type": "object",
        "properties": {
          "x": {
            "title": "X",
            "type": "integer"
          },
          "y": {
            "title": "Y",
            "type": "integer"
          }
        },
        "required": [
          "x",
          "y"
        ]
      }
    },
    {
      "name": "tan",
      "description": "Calculates the tangent of `theta`.",
      "parameters": {
        "type": "object",
        "properties": {
          "theta": {
            "title": "Theta",
            "type": "number"
          }
        },
        "required": [
          "theta"
        ]
      }
    }
  ],
  "function_call": "auto"
}

### Code Generation

*Marvin* offers an experimental utility to author code using OpenAI's function API. Given a function with a typed signature and a docstring, we can write entire functions in the language of your choice. Simple call `.code()` on a function decorated with @openai_fn. 

Behind the scenes, we define a utility function write_code:

In [ ]:
from marvin.openai import openai_fn

@openai_fn
def write_code(
    language: str,
    filename: str,
    name: str, 
    docstring: str, 
    code: str,
) -> str:
    '''Accepts and checks expertly staff engineer quality written `code` in `language`'''
    return(language, filename, name, docstring, code)

When you call the `code` method on *your* openai_fn, we simply call:

In [ ]:
@openai_fn
def add(x: int, y:int) -> str:
    '''Adds two numbers together'''
    # There is no code here! #

write_code.prompt([{
    'role': 'user',
    'content': f'''A function in python that described by the following schema:\n {add.schema}'''
}])

This returns the following payload for you to send to OpenAI, which forces it to write code that satisfies the high-level description of your function.

In [ ]:
{
    'messages': 
    [{
        'role': 'user',
        'content': "A function in python that described by the following schema:
            {'name': 'add', 
             'description': 'Adds two numbers together', 
             'parameters': {
                 'type': 'object', 
                 'properties': {
                     'x': {'title': 'X', 'type': 'integer'}, 
                     'y': {'title': 'Y', 'type': 'integer'}
                }, 'required': ['x', 'y']
            }
        }"
    }],
    'functions': [
        {
            'name': 'write_code',
            'description': 'Accepts and checks expertly staff engineer quality written `code` in `language`',
            'parameters': {
                'type': 'object',
                'properties': {
                    'language': {'title': 'Language', 'type': 'string'},
                    'filename': {'title': 'Filename', 'type': 'string'},
                    'name': {'title': 'Name', 'type': 'string'},
                    'docstring': {'title': 'Docstring', 'type': 'string'},
                    'code': {'title': 'Code', 'type': 'string'}},
            'required': ['language', 'filename', 'name', 'docstring', 'code']
            }
        }
    ],
    'function_call': {'name': 'write_code'}
}


If we give it a trivial example and call `.code()`, we see:

In [ ]:
from marvin.openai import openai_fn
import openai 
openai.api_key = 'YOUR_OPENAPI_KEY'

@openai_fn
def add(x: int, y:int) -> str:
    '''Adds two numbers together'''
    # There is no code here! #

write_code_instructions = add.code()

response = await openai.ChatCompletion.acreate(
    model = 'gpt-3.5-turbo', 
    **write_code_instructions
)

print(response.choices[0].message.get('function_call').get('arguments'))

Which returns

In [ ]:
{
  "language": "python",
  "filename": "add.py",
  "name": "add",
  "docstring": "Adds two numbers together",
  "code": "def add(x: int, y: int) -> int:\n    return x + y"
}